# Applied Data Science Capstone

<hr> 

## Week 3 Assignment: (Part 3) 
### Segmenting and Clustering Neighborhoods in Toronto
<hr>

In [1]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

raw_wiki= requests.get(wiki_url).text
soup = BeautifulSoup(raw_wiki,'html5lib')

In [3]:
table=soup.find('table')

In [4]:
Postcode= []
Borough= []
Neighbourhood = []

In [5]:
x=0 #counter
for table_cell in table.find_all('td'):
    
    x = x+1
    if (x==1):
        postal_code = table_cell.text
    elif (x==2):
        borough = table_cell.text
    elif (x==3):
        neighborhoods_data = table_cell.text
        neighborhoods = neighborhoods_data.strip('\n')
        x=0 #reset counter
                             
        if borough == 'Not assigned':            
            borough = []
            neighborhoods_data=[]
        else:
            if neighborhoods == 'Not assigned':
                neighborhoods = borough
             
            Postcode.append(postal_code)
            Borough.append(borough)
            Neighbourhood.append(neighborhoods)
                    

In [6]:
data=[]

for postcode_unique_element in set(Postcode):
    post_value = ''
    borough_value_var = ''
    neighbourhood_value = ''
    
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            post_value = postcode_element;
            borough_value_var = Borough[postcode_idx]
            
            if neighbourhood_value == '': 
                neighbourhood_value = Neighbourhood[postcode_idx]
            else:
                neighbourhood_value = neighbourhood_value + ', ' + Neighbourhood[postcode_idx]
    
    data.append([post_value,borough_value_var,neighbourhood_value])

df = pd.DataFrame(np.array(data),columns=['PostalCode', 'Borough', 'Neighborhood'])


<hr>

## PART 2

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
1,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ..."
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M3J,North York,"Northwood Park, York University"
4,M3N,North York,Downsview Northwest


In [8]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.columns = ['PostalCode', 'Latitude', 'Longitude']

In [9]:
df_join = pd.merge(df, df_latlng, on=['PostalCode'], how='inner')
df_join.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
1,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
3,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
4,M3N,North York,Downsview Northwest,43.761631,-79.520999


<hr>

## PART 3

In [10]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Clustered Map of Toronto

In [11]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



In [12]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

X = df_join['Latitude']
Y = df_join['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df_join['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df_join['Latitude'], df_join['Longitude'], df_join['Borough'], df_join['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map)  

map

### Now, you can notice how the region of Toronto neighborhood is segmented into 4 clusters in the above map